
## Merge the Lora adapter with the base model, save the merged model.

The merged model is saved in the save_dir, the Lora adapter is loaded from the lora_adapter_path, and these need to be specified.

Author: Matej Vadovic

Year: 2024


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from traintools.lib import predict, DatasetSingleton, MicroCoderTrain
import torch

# Path to the Lora adapter
lora_adapter_path = ''

# The directory to save the merged model 
save_dir = "microcoder-4b"

## Load model and tokenizer from HF

In [ ]:
config = CodeLLama7BConfig()

base_model = AutoModelForCausalLM.from_pretrained(
    config.model_name,
    torch_dtype=torch.float16,
    device_map=config.device_map,
    trust_remote_code=True,
    token=True,
    return_dict=True
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
tokenizer.pad_token = tokenizer.eos_token

## Merge adapter with model

In [ ]:
# Load PEFT model
MicroCoder = PeftModel.from_pretrained(base_model, lora_adapter_path)

# Merge base model and Lora adapter
MicroCoder = MicroCoder.merge_and_unload()

## Save new model

In [ ]:
# Save merged model
MicroCoder.save_pretrained(save_dir)

# Save tokenizera
tokenizer.save_pretrained(save_dir)

## Try model inference

In [ ]:
test_dataset = DatasetSingleton.get_test_dataset()

def build_prompt(docstring):
    text = f"{docstring}\n"
    return {'prompt_sample': text}

test_dataset = test_dataset.map(lambda x: build_prompt(x["prompt"]))

In [ ]:
# Generate predictions
example = f"{test_dataset['prompt_sample'][76]}\n"
prediction = predict(example, MicroCoder, tokenizer, temperature=0.8, repetition_penalty=1.3, skip_special_tokens=True)
print(prediction)